In [1]:
%reload_ext autoreload
%autoreload 2
from dragonn.tutorial_utils import *
%matplotlib inline

import numpy as np
import sys
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv


In [2]:
# Load keras libraries
print ('I am process:')
print (os.getpid())
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten, Activation, Dense, Dropout
from keras.utils import np_utils
from keras.callbacks import TensorBoard

import keras
import keras.backend as K
from keras.layers.core import ActivityRegularization
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

I am process:
6384


Using TensorFlow backend.


In [3]:
# Load sklearn libraries for evaluation purposes
from sklearn.utils import shuffle
import itertools
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn import metrics as me
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [46]:
# Converts DNA sequence into one_hot_encoding format
def get_training_input_data(input_file_path):
    x_train = []
    with open(input_file_path, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        reader.next()
        for num, sequence in reader:
            x_train.append(sequence)
    return x_train

def get_training_output_data(output_file_path):
    y_train = []
    with open(output_file_path, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        reader.next()
        for label in reader:
            y_train.append(label[1:])
    return y_train

def one_hot_encode(sequences):
    sequence_length = len(sequences[0])
    integer_type = np.int8 if sys.version_info[
        0] == 2 else np.int32  # depends on Python version
    integer_array = LabelEncoder().fit(np.array(('ACGTN',)).view(integer_type)).transform(
        sequences.view(integer_type)).reshape(len(sequences), sequence_length)
    one_hot_encoding = OneHotEncoder(
        sparse=False, n_values=5, dtype=integer_type).fit_transform(integer_array)

    return one_hot_encoding.reshape(
        len(sequences), sequence_length, 5, 1).swapaxes(1, 2)[:, [0, 1, 2, 4], :, :]
    #return one_hot_encoding.reshape(
    #    len(sequences), 1, sequence_length, 5).swapaxes(2, 3)[:, :, [0, 1, 2, 4], :]

In [47]:

# call train_test_split
# input_file_path = '/data/promoter_site/PromoterTrain.csv'
# output_file_path = '/data/promoter_site/SigmaTrain.csv'
input_file_path = '/data/promoter_site/smallTrainX.csv'
output_file_path = '/data/promoter_site/smallTrainY.csv'
x_train = get_training_input_data(input_file_path)
#print x_train.shape
y_train = get_training_output_data(output_file_path)
#print y_train
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

x_train = one_hot_encode(np.array(x_train))
print 'input shape: ' + str(x_train.shape)
x_val = one_hot_encode(np.array(x_val))
y_train = np.array(y_train).astype(bool)
y_val = np.array(y_val).astype(bool)
print 'output shape: ' + str(y_train.shape)


input shape: (75, 4, 51, 1)
output shape: (75, 5)


In [94]:
from dragonn.models import SequenceDNN
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.layers import Embedding
from keras.layers import Conv2D, GlobalMaxPooling2D
from keras import regularizers

#model = SequenceDNN(seq_length=51, num_tasks=1, num_filters=[15],conv_width=[15], pool_width=15, dropout=0.1)
#model.train(x_train, y_train, validation_data=(x_val, y_val))

num_filters=(5, 5, 5)
conv_width=(3, 15, 15)
pool_width=15
seq_length = 51
model = Sequential()
for i, (nb_filter, nb_col) in enumerate(zip(num_filters, conv_width,)):
    conv_height = 4 if i == 0 else 1
    model.add(Conv2D(nb_filter, conv_height, nb_col, \
                     activation='relu', input_shape=(4,seq_length,1), \
                    kernel_regularizer=regularizers.l2(0.01), \
                    bias_regularizer=regularizers.l2(0.01)))
    # BatchNormalization doesn't help reduce loss
    # model.add(BatchNormalization())
    print model.output_shape
    model.add(Dropout(0.25))

#model.add(Conv2D(15, 4, 3, activation='relu', input_shape=(4,51,1)))
#print model.output_shape
#model.add(Dropout(0.25))
#model.add(MaxPooling2D(pool_size=(1, pool_width)))
print model.output_shape
model.add(Flatten())
print model.output_shape
model.add(Dense(output_dim=5))
print model.output_shape
model.add(Activation('softmax'))
print model.output_shape
model.compile(optimizer='adam', loss='binary_crossentropy')

model.fit(x_train, y_train, epochs=20, batch_size=12, validation_data=(x_val, y_val))

#valid_result = model.test(x_val, y_val)
#print valid_result

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (4, 3), activation="relu", kernel_regularizer=<keras.reg..., input_shape=(4, 51, 1), bias_regularizer=<keras.reg...)`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (1, 15), activation="relu", kernel_regularizer=<keras.reg..., input_shape=(4, 51, 1), bias_regularizer=<keras.reg...)`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=5)`


(None, 1, 49, 5)
(None, 1, 35, 5)
(None, 1, 21, 5)
(None, 1, 21, 5)
(None, 105)
(None, 5)
(None, 5)
Train on 75 samples, validate on 25 samples
Epoch 1/20
75/75 [==============================] - 1s 16ms/step - loss: 0.6571 - val_loss: 0.6439
Epoch 2/20
75/75 [==============================] - 0s 328us/step - loss: 0.6365 - val_loss: 0.6284
Epoch 3/20
75/75 [==============================] - 0s 326us/step - loss: 0.6175 - val_loss: 0.6075
Epoch 4/20
75/75 [==============================] - 0s 329us/step - loss: 0.5923 - val_loss: 0.5764
Epoch 5/20
75/75 [==============================] - 0s 328us/step - loss: 0.5678 - val_loss: 0.5270
Epoch 6/20
75/75 [==============================] - 0s 336us/step - loss: 0.5374 - val_loss: 0.4865
Epoch 7/20
75/75 [==============================] - 0s 359us/step - loss: 0.5129 - val_loss: 0.4731
Epoch 8/20
75/75 [==============================] - 0s 357us/step - loss: 0.5121 - val_loss: 0.4737
Epoch 9/20
75/75 [==============================] - 0s 35

In [ ]:
input_shape = (_, 1, 4, 51)
nclass = 32 # Five different sigma factors, present or absent -> 2^5

In [ ]:
def CNN_model():
    

In [96]:
sequences = ['GTCACC','GCCCAT']
seq = np.array(sequences)
print(np.array(seq[0]).shape)
print(len(seq))
print(seq.shape)
result = one_hot_encode(np.array(sequences))
print(LabelEncoder().fit(np.array(('ACGTN',)).view(np.int8)))
integer_type = np.int8
integer_array = LabelEncoder().fit(np.array(('ACGTN',)).view(integer_type)).transform(seq.view(integer_type))
print(integer_array.shape)
integer_array = integer_array.reshape(len(sequences), 6)
print(integer_array)

one_hot_encoding = OneHotEncoder(sparse=False, n_values=5, dtype=integer_type).fit_transform(integer_array)
print(one_hot_encoding)

new = one_hot_encoding.reshape(len(sequences), 1, 6, 5).swapaxes(2, 3)[:, :, [0, 1, 2, 4], :]
print(new)
print(new.shape)


()
2
(2,)
LabelEncoder()
(12,)
[[2 4 1 0 1 1]
 [2 1 1 1 0 4]]
[[0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1]]
[[[[0 0 0 1 0 0]
   [0 0 1 0 1 1]
   [1 0 0 0 0 0]
   [0 1 0 0 0 0]]]


 [[[0 0 0 0 1 0]
   [0 1 1 1 0 0]
   [1 0 0 0 0 0]
   [0 0 0 0 0 1]]]]
(2, 1, 4, 6)


In [24]:
print(result)

[[[[0 0 0 1 0 0]
   [0 0 1 0 1 1]
   [1 0 0 0 0 0]
   [0 1 0 0 0 0]]]


 [[[0 0 0 0 1 0]
   [0 1 1 1 0 0]
   [1 0 0 0 0 0]
   [0 0 0 0 0 1]]]]


In [25]:
print(result.shape)

(2, 1, 4, 6)


In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np, random
np.random.seed(1)
random.seed(1)
from dragonn.models import SequenceDNN
from simdna.simulations import simulate_single_motif_detection
from dragonn.utils import one_hot_encode, get_motif_scores, reverse_complement
try:
    from sklearn.model_selection import train_test_split  # sklearn >= 0.18
except ImportError:
    from sklearn.cross_validation import train_test_split  # sklearn < 0.18
import sys

# Settings

seq_length = 500
num_sequences = 8000
num_positives = 4000
num_negatives = num_sequences - num_positives
GC_fraction = 0.4
test_fraction = 0.2
validation_fraction = 0.2
do_hyperparameter_search = False
num_hyperparameter_trials = 50
num_epochs = 100
use_deep_CNN = False
use_RNN = False

print('Generating sequences...')

sequences, labels, embeddings = simulate_single_motif_detection(
    'SPI1_disc1', seq_length, num_positives, num_negatives, GC_fraction)

print('One-hot encoding sequences...')

encoded_sequences = one_hot_encode(sequences)

print('Getting motif scores...')

motif_scores = get_motif_scores(encoded_sequences, motif_names=['SPI1_disc1'])

print('Partitioning data into training, validation and test sets...')

X_train, X_test, y_train, y_test = train_test_split(encoded_sequences, labels, test_size=test_fraction)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_fraction)

print('Adding reverse complements...')

X_train = np.concatenate((X_train, reverse_complement(X_train)))
y_train = np.concatenate((y_train, y_train))

print('Randomly splitting data into training and test sets...')

random_order = np.arange(len(X_train))
np.random.shuffle(random_order)
X_train = X_train[random_order]
y_train = y_train[random_order]

# Build and train model

if not do_hyperparameter_search:
    hyperparameters = {'seq_length': seq_length, 'use_RNN': use_RNN,
                       'num_filters': (45,), 'pool_width': 25, 'conv_width': (10,),
                       'L1': 0, 'dropout': 0.2, 'num_epochs': num_epochs}
    if use_deep_CNN:
        hyperparameters.update({'num_filters': (45, 50, 50), 'conv_width': (10, 8, 5)})
    if use_RNN:
        hyperparameters.update({'GRU_size': 35, 'TDD_size': 45})
    model = SequenceDNN(**hyperparameters)
    model.train(X_train, y_train, validation_data=(X_valid, y_valid),
                save_best_model_to_prefix='best_model')

else:
    print('Starting hyperparameter search...')
    from dragonn.hyperparameter_search import HyperparameterSearcher, RandomSearch
    fixed_hyperparameters = {'seq_length': seq_length, 'use_RNN': use_RNN, 'num_epochs': num_epochs}
    grid = {'num_filters': ((5, 100),), 'pool_width': (5, 40),
            'conv_width': ((6, 20),), 'dropout': (0, 0.5)}
    if use_deep_CNN:
        grid.update({'num_filters': ((5, 100), (5, 100), (5, 100)),
                     'conv_width': ((6, 20), (6, 20), (6, 20))})
    if use_RNN:
        grid.update({'GRU_size': (10, 50), 'TDD_size': (20, 60)})

    # Backend is RandomSearch; if using Python 2, can also specify MOESearch
    # (requires separate installation)
    searcher = HyperparameterSearcher(SequenceDNN, fixed_hyperparameters, grid, X_train, y_train,
                                      validation_data=(X_valid, y_valid), backend=RandomSearch)
    searcher.search(num_hyperparameter_trials)
    print('Best hyperparameters: {}'.format(searcher.best_hyperparameters))
    model = searcher.best_model

# Test model

print('Test results: {}'.format(model.test(X_test, y_test)))

# Plot DeepLift and ISM scores for the first 10 test examples, and model architecture

if sys.version[0] == 2:
    model.plot_deeplift(X_test[:10], output_directory='deeplift_plots')
model.plot_in_silico_mutagenesis(X_test[:10], output_directory='ISM_plots')
model.plot_architecture(output_file='architecture_plot.png')